# E8.아이유팬이 좋아할 만한 다른 아티스트 찾기

`28 JAN 2020`

-----------------

>1. 데이터 준비와 전처리
>2. 분석해 봅시다.
>3. 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.
>4. CSR matrix를 직접 만들어 봅시다.
>5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.
>6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
>7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
>8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.


## 8-9. 프로젝트 - Movielens 영화 추천 실습


이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.
이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
별점을 시청횟수로 해석해서 생각하겠습니다.

또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
MovieLens 1M Dataset 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축해제 하겠습니다.

1) wget으로 데이터 다운로드
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 옮김
$ mv ml-1m.zip ~/aiffel/recommendata_iu/data

3) 작업디렉토리로 이동
$ cd ~/aiffel/recommendata_iu/data

4) 압축 해제
$ unzip ml-1m.zip

### 필요한 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
import os
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

### 1) 데이터 준비와 전처리


Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.



In [2]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')

orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [5]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [6]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [7]:
# 사용하는 컬럼 재정의
using_cols = ['user_id', 'movie_id', 'count']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [8]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


- [Pandas-df 합치기](https://antilibrary.org/2483)


df을 합치는 방법은 여러가지가 있다.

concat(), merge(), join()방법을 찾았다. 

데이터들을 합칠때 신중하게 잘 진행해야한다. 잘못 진행하면 이상한 방향으로 흘러들어간다...

1. 데이터를 합칠때 어떤 기준으로 합쳐야 하는지의 설정이 중요하다.

- title, movie_id 등등.. 이부분에서 아직 부족하다..

- 여러번의 시도 끝에 일단 이러한 결과를 얻었지만 여러 데이터를 가지고 연습이 필요하다고 생각된다. 

2. 유의미한 데이터를 남겨놓는 작업도 중요하다.

- 필요없는 데이터는 drop시키거나 내가 사용할 데이터를 가지고 작업할 줄 알아야한다.

- 데이터를 남길때 원본 유지는 필수! 

In [9]:
df = movies.set_index("movie_id")
ratings = ratings.join(df, on = "movie_id")

ratings

,user_id,movie_id,count,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,My Fair Lady (1964),Musical|Romance
3,1,3408,4,Erin Brockovich (2000),Drama
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...
1000203,6040,1090,3,Platoon (1986),Drama|War
1000205,6040,1094,5,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,Sophie's Choice (1982),Drama


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

데이터 분석은 전처리 과정이 절반 이상이라고 해도 과언이 아니다.

전처리 과정을 탄탄하게 해놔야 이후의 작업도 탄탄대로라고 할 수 있다.

연습만이 살길이다...!

### 2) 분석해 봅시다.

- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [10]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [11]:
#ratings['title'].nunique()

In [12]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [13]:
# 가장 인기 있는 영화 30개(인기순) 
popular_movie = ratings.groupby('title')['user_id'].count()

popular_movie.sort_values(ascending=False).head(30) # 위에 count()뒤에 연결가능 

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

가장 인기있는 영화 30개를 보고싶은데 title이 이상하게 변수로 작용을 하지 않았다.

그래서 일단 movie_id와 count를 사용했고 계속해서 영화 이름이 나타나도록 시도해보았다. 

-> df을 제대로 합치지 않아 발생한 문제였다.(+인덱싱도 잘못했었다.)



In [14]:
movie_ratings = ratings.groupby("movie_id")["count"].mean()

### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.


원래 나홀로집에 1,2,3, 백투더퓨쳐 1,2 이 다섯가지로 골라서 rating에 추가했더니 결과가 영... 파이라서 두번째 시도에는 각기 다른영화로 추가해보았습니다.

`my_favorite = [
'Home Alone (1990)','Home Alone 2: Lost in New York (1992)','Home Alone 3 (1997)',
'Back to the Future (1985)','Back to the Future Part II (1989)' ]`

제가 좋아하는 영화들이 있는지 단어들로 찾아보려고 하였습니다.

아쉽게도 노트북 영화는 없네요. 다행히 최애 영화들인 나홀로집에, 백투더퓨처는 있군요! 

In [15]:
#movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 

#영화 제목을 인덱싱하는 작업

idx_to_movie = {i:v for i,v in zip(movies['movie_id'], movies['title'])}
movie_to_idx = {v:i for i,v in zip(movies['movie_id'], movies['title'])}

In [23]:
# 원하는 영화 찾기 방법
movie = ratings[ratings['title'].str.contains("Future")]
print(movie['title'].unique())

['Back to the Future (1985)' 'X-Files: Fight the Future, The (1998)'
 'Back to the Future Part II (1989)' 'Back to the Future Part III (1990)']


In [16]:
search_words = ["Alone", "Future"]  # 원하는 단어로 영화 찾아보기 

for key, item in idx_to_movie.items():
    for word in search_words:
        if word in item:
            print(key, item)

586 Home Alone (1990)
1270 Back to the Future (1985)
1707 Home Alone 3 (1997)
1909 X-Files: Fight the Future, The (1998)
2011 Back to the Future Part II (1989)
2012 Back to the Future Part III (1990)
2557 I Stand Alone (Seul contre tous) (1998)
2953 Home Alone 2: Lost in New York (1992)


In [17]:
# 본인이 좋아하시는 영화 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = [
'Matilda (1996)','Home Alone 2: Lost in New York (1992)','Titanic (1997)',
'Back to the Future (1985)','Parent Trap, The (1998)' ]


In [18]:
my_idx = []
for i in my_favorite:
    my_idx.append(movie_to_idx[i])
my_idx

[837, 2953, 1721, 1270, 2059]

In [19]:

my_movielist = pd.DataFrame({'user_id': ['ha0']*5, 'movie_id' : my_idx, 'title': my_favorite, 'count':[5]*5})

if not ratings.isin({'user_id':['ha0']})['user_id'].any():  # user_id에 'ha0'이라는 데이터가 없다면
    ratings = ratings.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.


,user_id,movie_id,count,title,genre
1000203,6040,1090,3,Platoon (1986),Drama|War
1000205,6040,1094,5,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,Sophie's Choice (1982),Drama
1000208,6040,1097,4,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,ha0,837,5,Matilda (1996),NaN
1,ha0,2953,5,Home Alone 2: Lost in New York (1992),NaN
2,ha0,1721,5,Titanic (1997),NaN
3,ha0,1270,5,Back to the Future (1985),NaN
4,ha0,2059,5,"Parent Trap, The (1998)",NaN


In [24]:
#user를 찾아내보자
user_unique = ratings['user_id'].unique()

#user를 indexing 해보자
user_to_idx = {v:k for k,v in enumerate(user_unique)}

In [25]:
#movie를 찾아내보자
movie_unique = ratings['title'].unique()

#movie를 indexing 해보자
movie_to_idx = {v:i for i,v in enumerate(movie_unique)}

In [26]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다

temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 준다
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 준다
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')
ratings.tail(10)

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count,title,genre
1000203,6038,1030,3,Platoon (1986),Drama|War
1000205,6038,986,5,"Crying Game, The (1992)",Drama|Romance|War
1000206,6038,311,5,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6038,142,4,Sophie's Choice (1982),Drama
1000208,6038,26,4,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,6039,1420,5,Matilda (1996),NaN
1,6039,2015,5,Home Alone 2: Lost in New York (1992),NaN
2,6039,27,5,Titanic (1997),NaN
3,6039,22,5,Back to the Future (1985),NaN
4,6039,1510,5,"Parent Trap, The (1998)",NaN


### 4) CSR matrix를 직접 만들어 봅시다.

user의 수는 원래 6039였는데 'ha0'인 내가 한명 추가되었으므로 6040

영화 수는 원래 영화에서 5가지를 골랐으므로 그대로이다.

In [27]:
num_user = ratings['user_id'].nunique()
num_user

6040

In [28]:
num_movie = ratings['movie_id'].nunique()
num_movie

3628

In [29]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.


In [30]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분이다
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

모델 정규화 같은 속성들을 바꾸면 더 추천이 좋아질까?

In [31]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [32]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

내가 선호하는 영화와 그 외 영화 토이스토리로 선호도를 파악해보았다.



In [33]:
ha0, Toy_story = user_to_idx['ha0'], movie_to_idx['Toy Story (1995)']
ha0_vector, Toy_story_vector = als_model.user_factors[ha0], als_model.item_factors[Toy_story]

print('완료')

완료


In [34]:
ha0_vector

array([-1.31448045e-01, -5.70828676e-01,  2.31556803e-01,  1.64574072e-01,
       -1.70750506e-02,  1.15111202e-01,  2.52591312e-01,  7.25793421e-01,
       -4.42769766e-01, -1.65996123e-02,  2.75488943e-01, -3.41687799e-01,
        1.36914283e-01,  5.12207687e-01,  1.43818304e-01,  2.26297736e-01,
        1.67233512e-01, -9.98626947e-01, -3.19108546e-01, -2.25706808e-02,
       -5.33928812e-01, -1.78330123e-01, -4.39474992e-02, -7.91481659e-02,
       -3.12378109e-01,  5.01984894e-01,  8.78145576e-01,  2.57453531e-01,
        2.30323210e-01,  7.28077471e-01, -4.05966908e-01,  1.11392474e+00,
        1.43844843e-01,  7.64631212e-01,  1.96077600e-01, -5.87313287e-02,
        3.34935248e-01, -3.85760427e-01,  4.87732291e-01,  1.12002574e-01,
        7.17818320e-01, -6.51513100e-01, -4.71388698e-01,  1.43757179e-01,
       -3.78253490e-01, -1.14670433e-01,  4.95349944e-01,  5.65722361e-02,
        9.21890199e-01, -1.68329388e-01,  1.85681701e-01,  5.56915879e-01,
       -2.70947833e-02, -

In [35]:
Toy_story_vector

array([ 0.02558637,  0.01878715,  0.02764802,  0.01529729,  0.00619644,
        0.04858719,  0.00113384,  0.00121566,  0.00411019,  0.03423275,
        0.00090049,  0.03593095, -0.03003642, -0.00036406,  0.02057206,
        0.0313094 ,  0.00292168, -0.00903727,  0.0102788 ,  0.00729911,
       -0.02760817, -0.00519236, -0.02191708, -0.02942139,  0.00416615,
       -0.00245282,  0.03035082,  0.0116144 ,  0.00156674,  0.02075652,
        0.00342146,  0.03392329, -0.00775286,  0.02303392,  0.00953288,
        0.00756566,  0.00417457,  0.01049322, -0.00351026, -0.02869998,
        0.01772414,  0.00212329, -0.0042088 , -0.02258035,  0.01320579,
        0.01384765, -0.01930784,  0.01851452,  0.02116745,  0.01446293,
       -0.00084562,  0.01143232,  0.01670431,  0.01513993, -0.00957925,
       -0.01824728,  0.02277324,  0.00282697,  0.03406526,  0.02094662,
        0.00342606,  0.00010768, -0.02807985,  0.0283278 , -0.04035931,
        0.00482651, -0.03285234, -0.02197231,  0.0153986 ,  0.04

In [36]:
np.dot(ha0_vector, Toy_story_vector )

0.13572058

추천도가 0.13이라니.. 

ET는 조금더 높게 나오긴했다. 

In [37]:
ET = movie_to_idx['E.T. the Extra-Terrestrial (1982)']
ET_vec = als_model.item_factors[ET]
np.dot(ha0_vector, ET_vec)


0.26577014

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.


In [38]:
# AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 아티스트를 찾는다.
# 처음으로는 Pulp Fiction에 대해서 찾아보겠다

favorite_movie = 'Matilda (1996)'

movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)

[idx_to_movie[i[0]] for i in similar_movie]

['Message to Love: The Isle of Wight Festival (1996)',
 'Friday the 13th Part VIII: Jason Takes Manhattan (1989)',
 'Gremlins (1984)',
 'Touch of Evil (1958)',
 'Band Wagon, The (1953)',
 'Dune (1984)',
 'Apple Dumpling Gang Rides Again, The (1979)',
 'Wonderful, Horrible Life of Leni Riefenstahl, The (Die Macht der Bilder) (1993)',
 'Exorcist III, The (1990)',
 'Friday the 13th Part V: A New Beginning (1985)',
 'Prom Night IV: Deliver Us From Evil (1992)',
 'Poltergeist III (1988)',
 'Spellbound (1945)',
 'Topaz (1969)',
 'Year of Living Dangerously (1982)']

In [39]:
# 아티스트의 id, 유사도를 Tuple로 반환하고 있다. 그렇기 때문에 아티스트의 id를 다시 아티스트의 이름으로 매핑 시켜 준다.
# artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성

idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Matilda (1996)',
 'Harriet the Spy (1996)',
 'Madeline (1998)',
 'Mouse Hunt (1997)',
 'Casper (1995)',
 'Theodore Rex (1995)',
 'Dunston Checks In (1996)',
 'Paulie (1998)',
 'Air Bud (1997)',
 'Andre (1994)',
 'Buddy (1997)',
 'George of the Jungle (1997)',
 'Hocus Pocus (1993)',
 'Fluke (1995)',
 'First Kid (1996)']

In [40]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

# 몇 번 더 반복해서 확인하기 위해 위의 코드를 함수로 만들고 확인해보겠다
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [41]:
get_similar_movie('Back to the Future (1985)')

['Back to the Future (1985)',
 "Ferris Bueller's Day Off (1986)",
 'Back to the Future Part II (1989)',
 'When Harry Met Sally... (1989)',
 'Big (1988)',
 'Fish Called Wanda, A (1988)',
 'E.T. the Extra-Terrestrial (1982)',
 'Ghostbusters (1984)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode IV - A New Hope (1977)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

오 ET영화가 0.266으로 가장 높은 추천도를 가졌네요?!


In [42]:
user = user_to_idx['ha0']
# recommend에서는 user*item CSR Matrix를 받습니다.

movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
for i in movie_recommended: 
    print("{} {:0.3f}".format(idx_to_movie[i[0]], i[1]))

E.T. the Extra-Terrestrial (1982) 0.266
Galaxy Quest (1999) 0.259
Christmas Story, A (1983) 0.225
Toy Story 2 (1999) 0.212
Ferris Bueller's Day Off (1986) 0.204
Star Wars: Episode V - The Empire Strikes Back (1980) 0.197
Star Wars: Episode VI - Return of the Jedi (1983) 0.194
Star Wars: Episode IV - A New Hope (1977) 0.191
American Beauty (1999) 0.191
Casper (1995) 0.186
When Harry Met Sally... (1989) 0.182
Ghostbusters (1984) 0.175
Forrest Gump (1994) 0.172
Secret Garden, The (1993) 0.172
You've Got Mail (1998) 0.171
Star Wars: Episode I - The Phantom Menace (1999) 0.170
Airplane! (1980) 0.158
Home Alone (1990) 0.157
Back to the Future Part II (1989) 0.157
101 Dalmatians (1996) 0.156


## 결과 정리

- 내가 좋아하는 영화 5:

Matilda (1996)   
Home Alone 2: Lost in New York (1992)   
Titanic (1997)   
Back to the Future (1985)   
Parent Trap, The (1998)    

- 추천받은 내가 좋아할 만한 영화들 Top5:

E.T. the Extra-Terrestrial (1982) 0.266   
Galaxy Quest (1999) 0.259   
Christmas Story, A (1983) 0.225   
Toy Story 2 (1999) 0.212   
Ferris Bueller's Day Off (1986) 0.204   

## 어려웠던 점

- 선호도가 생각보다 너무 낮게 나왔다 
1. 처음에 내가 선택한 영화와, 그외의 영화를 선택해서 계산했다. 나의 실수~!

2. 선호도가 생각보다 높게 안나와서 추천영화에있는 ET를 넣어서 계산해보았는데 그래도 결과값은 0.26정도
내 모델이 잘못인가...?

- 처음에는 나홀로집에1,2,3 백투더 퓨처1,2 이 다섯가지 영화로 넣었는데 추천 선호도 결과들이 0.05,0.01이렇게 나와서 너무 시리즈들로만 해서 모델이 추천하기 어렵나...? 이렇게 생각이 들어서 두번째에는 여러 영화를 나름 내가 섞어서 넣었다. 


- 좋아할 만한 영화들의 결과를 보았을때 오 추천이 잘 되는것 같다는 생각이 들었다. 다만 선호도 결과값이 생각했던 것보다 낮게 나와서 모델에 문제가 있거나 아님 내가좋아하는 영화가 너무 다양한 분야라 그런가 싶었다. 


- 이 추천시스템에 대한 신뢰도,정확도를 높이려면 어떻게 해야할까?


- 일관된 사용자의 성향데이터 vs 여러 분야를 좋아하는 사용자의 데이터 둘중에 어떤 부분이 추천시스템의 모델의 정확도를 더 높게 가지고 갈 수 있을까?


- 데이터를 합치는데 처음에 잘못 선정해서 중간에 에러의 늪에 빠져있었다. 또한 좋아하는 영화선정해서 넣을때도 오류가 많았는데 여러번 시도 후 해결 할 수 있었다.

## 느낀점

- 추천시스템 재미는 있었는데 아직 명확히 이론이 잘 이해는 안간다. (CRS Matrix부분)

- 은행의 주식, 상품추천도 이런 방식으로 이루어지겠지? 사용자의 소비 패턴, 주식 선정, 주식 종류, 매도, 매수방법등 정말 많은 데이터를 보유 하여 주식상품 추천, 은행 상품 추천도 이루어질 것 같다. 이방법에 대해서 자세히 알아보고 싶어졌다.  




### 부족한 개념 공부하자!

- Pandas DataFrame 활용
- 추천시스템의 개념

- CS